In [135]:
import pandas as pd
import re
import matplotlib.pyplot as plt
from ipywidgets import interact, Layout, IntSlider
import ipywidgets as ipw
import timeit



xl = 'income.xlsx'
df = pd.read_excel(xl)


ink_intervall, ink_klasser, klass_storlek, klass_index, åld_grupp = [],[],[],[],0
less_you, more_you, lägre_klass, procent, topp = 0,0,0,0,0

########################################################################
###########################  Data Frame  ###############################
########################################################################

tot_bef, alla_inkomster, åldrar = df.iloc[3:88,8,].tolist(), df.iloc[173:2298,8,].tolist(), df.iloc[3:88,1,].tolist()
df1, df1.index = pd.DataFrame(), åldrar


#loopar och delar upp i inkomstklasser åldrar 16-100+, ersätter '..' med 0
for c in range(0,len(alla_inkomster),85):
    temp = [0 if alla_inkomster[c+cc] == '..' else alla_inkomster[c+cc] for cc in range(0,85)]
    ink_klasser.append(temp)


#summera hur stor inkomstklassen är
#for i in ink_klasser: klass_storlek.append(sum(i))
klass_storlek = [ sum(i) for i in ink_klasser]


#loopar in data i kolumner & döper kolumner
for (i,ii) in zip(list(range(0,400,20)), list(range(20))): df1[f"""{i}-{i+19} tkr""" ] = ink_klasser[ii]
for (i,ii) in zip(list(range(400,600,100)), list(range(20,22))): df1[f"""{i}-{i+99} tkr""" ] = ink_klasser[ii]
for (i,ii) in zip(list(range(600,1000,200)), list(range(22,24))): df1[f"""{i}-{i+199} tkr""" ] = ink_klasser[ii]


#lägger till sista kolumn + rad & döper om första kolumn manuellt
df1['1000+ tkr'],df1.loc['Antal'] = ink_klasser[-1],klass_storlek
df1.rename(columns = {'0-19 tkr':'1-19 tkr'}, inplace=True)



########################################################################
###########################  Input  ####################################
########################################################################


def löne_index(m_lön):
    
    
    global ink_intervall

    års_inkomst = m_lön*12

    
    #splitar index kolumnnamn med re.split(), filtrerar bort '' + omvandlar till int i ny lista
    for cols in df1:
        ink_range = list(filter(None, re.split("-| tkr|\+ tkr| [\s_+]| ", cols)))


        if len(ink_range) == 1: #testar om superrik
            ink_intervall = '1000+ tkr' #hitta inkomstintervall
            break

        elif int(ink_range[0]) <= års_inkomst < int(ink_range[1]): #letar upp intervall
            ink_intervall = f"""{ink_range[0]}-{ink_range[1]} tkr""" 
            break

            
    klass_index = df1.columns.get_loc(ink_intervall) #hitta intervall index
    return klass_index


def curiosa(m_lön, ålder):

    
    global less_you, more_you, procent, topp, åld_grupp, lägre_klass
    t1,t2,t3 = 0,0,0
     
        
    klass_index = löne_index(m_lön)
    åld_grupp = df1.iat[ålder-16, klass_index]

    
    for a in range(0, klass_index): t1 += klass_storlek[a] #summer storleken
    
    
    for b in range(0,klass_index): t2 += df1.iat[ålder-16,b] #summerar alla i åldern som tjänar mindre
    for c in range(klass_index+1, 25): t3 += df1.iat[ålder-16,c] #summerar alla i åldern som tjänar mindre

    
    lägre_klass,less_you,more_you = t1,t2,t3    
    procent = lägre_klass/sum(tot_bef)*100
    topp = 100-procent


########################################################################
###########################  Output  ###################################
########################################################################

def textruta():

    
    p1 = (f"""
    Inkomstklass: {ink_intervall}/år.

    I din ålder befiner sig {more_you:,} i högre inkomstklass. {less_you:,} i lägre.
    {åld_grupp:,} stycken i din inkomstklass är lika gamla.
    
    Du tillhör topp {topp:.1f}% av inkomsttagare.
    """ )

    return p1


def visualisering(Ålder,klass_index):

    
    p = textruta()
    
    
    #väljer den högsta av inkomsterklasser, färglägger kolumner
    topp_klass = max(df1.iloc[Ålder-16,:].tolist()) #
    stolpar = df1.iloc[Ålder-16,:] #ålder, x=inkomstklass koordinater
    colors = [f"b" for i in range(25)]
    colors[klass_index] = 'g'

    
    #riket = df1.iloc[85,0:]   för hela riket

    
    
    plt.text(5,topp_klass*0.63, p, fontsize=12,
            bbox = dict(facecolor = 'white', alpha=0.5))
    

    
    bakgrund = stolpar.T.plot.bar(color = colors,figsize=(23,7));
    bakgrund.set_facecolor('black')

    
########################################################################
#############################  Interaction  ############################
########################################################################



def graf(Ålder, Månadslön):
    
    curiosa(Månadslön, Ålder)
    index = löne_index(Månadslön)
    visualisering(Ålder, index)


#display(df1)


interact(graf,  Månadslön = IntSlider(25,1,100,1,layout=Layout(width='500px')),
                Ålder = IntSlider(34,16,100,1,layout=Layout(width='500px')));




interactive(children=(IntSlider(value=34, description='Ålder', layout=Layout(width='500px'), min=16), IntSlide…

In [68]:
#import ipywidgets as ipw
#from ipywidgets import interact, Layout, IntSlider



#@interact( Månadslön = (1,100), Ålder = (16,100))
#def graf(Ålder, Månadslön):
 #   pass


#interact(graf, Månadslön=IntSlider(50,10,100,1,layout=Layout(width='500px')), Ålder=IntSlider(0,25,100,1,layout=Layout(width='500px')));
